#V2

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Reshape, LSTM, Dense
import matplotlib.pyplot as plt

# Load the data
data = pd.read_csv('Combined_PTF04_Revised.csv', low_memory=False)

# Parse datetime column with errors='coerce' to handle inconsistent formats
data['datetime'] = pd.to_datetime(data['datetime'], errors='coerce')

# Drop rows with NaT in datetime column
data = data.dropna(subset=['datetime'])

# Function to preprocess data for a specific city
def preprocess_data(city_name, data, sequence_length=14):
    # Filter data for the specified city
    city_data = data[data['city_name'] == city_name].sort_values('datetime')

    # Select relevant columns and drop missing values
    city_data = city_data[['datetime', 'main.aqi', 'components.co', 'components.no2', 'components.o3', 'components.so2', 'components.pm2_5', 'components.pm10']].dropna()

    # Normalize the data
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(city_data.iloc[:, 1:])

    # Create sequences
    X, y = [], []
    for i in range(len(scaled_data) - sequence_length):
        X.append(scaled_data[i:i+sequence_length])
        y.append(scaled_data[i+sequence_length])

    X = np.array(X)
    y = np.array(y)

    return X, y, scaler, city_data

# Reshape X for the Conv1D layer (requires 3D input)
def reshape_input(X):
    return X.reshape((X.shape[0], X.shape[1], X.shape[2]))

# Define the model
def create_cnn_lstm_model(input_shape):
    model = Sequential()
    model.add(Conv1D(32, 2, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(2))
    model.add(Flatten())
    model.add(Reshape((-1, 32)))
    model.add(LSTM(50, activation='relu'))
    model.add(Dense(7))
    model.compile(optimizer='adam', loss='mse')
    return model

# Function to make predictions for today based on the latest sequence
def predict_future_aqi(city_name, data, model, scaler, sequence_length=14, future_days=7):
    X, _, _, city_data = preprocess_data(city_name, data, sequence_length)
    X_reshaped = reshape_input(X)

    # Get the latest sequence to make future predictions
    latest_sequence = X_reshaped[-1].reshape(1, sequence_length, -1)

    predictions = []
    for _ in range(future_days):
        prediction = model.predict(latest_sequence)
        predictions.append(prediction[0])

        # Append the prediction to the latest_sequence and remove the oldest entry
        latest_sequence = np.append(latest_sequence[:, 1:, :], prediction.reshape(1, 1, -1), axis=1)

    predictions = scaler.inverse_transform(predictions)

    return predictions


# Function to plot today's prediction
def plot_future_predictions(predictions, city_name):
    pollutants = ['main.aqi', 'components.co', 'components.no2', 'components.o3', 'components.so2', 'components.pm2_5', 'components.pm10']
    future_days = len(predictions)

    fig, axes = plt.subplots(len(pollutants), 1, figsize=(10, 18))
    fig.suptitle(f"Future AQI Predictions for {city_name}", fontsize=16)

    for i, pollutant in enumerate(pollutants):
        axes[i].plot(range(future_days), predictions[:, i], marker='o', linestyle='-')
        axes[i].set_ylabel(pollutant)
        axes[i].set_xlim(0, future_days-1)
        axes[i].set_xticks(range(future_days))
        axes[i].set_xticklabels([f'Day {j+1}' for j in range(future_days)])

        # Annotate the points with the predicted values
        for j in range(future_days):
            axes[i].annotate(f'{predictions[j, i]:.2f}', (j, predictions[j, i]), textcoords="offset points", xytext=(0,10), ha='center')

    plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust layout to make room for the title
    plt.show()




# Get the list of available cities in the dataset
available_cities = data['city_name'].unique()

# Prompt the user for a valid city name
while True:
    city_name = input("Enter the city name: ")
    if city_name in available_cities:
        break
    else:
        print(f"City '{city_name}' not found in the dataset. Please try again.")

# Preprocess data for the specified city and create the model
X, y, scaler, _ = preprocess_data(city_name, data)
X_reshaped = reshape_input(X)
input_shape = (X_reshaped.shape[1], X_reshaped.shape[2])
model = create_cnn_lstm_model(input_shape)

# Train the model on all data
model.fit(X_reshaped, y, epochs=10, batch_size=32, validation_split=0.2)

# Predict the future AQI for the next 7 days
future_predictions = predict_future_aqi(city_name, data, model, scaler, sequence_length=14, future_days=7)

# Plot the future predictions
plot_future_predictions(future_predictions, city_name)



KeyboardInterrupt: Interrupted by user

#Anvil

In [ ]:
!pip install anvil-uplink pandas scikit-learn tensorflow

import anvil.server
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Reshape, LSTM, Dense

# Connect to Anvil
anvil.server.connect("server_DYNDZNG3KBNFJNT3EUQTKTQ6-PLY2MVWZBUVGYSMJ")

# Load the data directly from the Colab environment
data = pd.read_csv('Combined_PTF04_Revised.csv', low_memory=False)
data['datetime'] = pd.to_datetime(data['datetime'], errors='coerce')
data = data.dropna(subset=['datetime'])

# Extract unique city names
unique_city_names = list(data['city_name'].unique())

scaler = None
model = None

def preprocess_data(city_name, data, sequence_length=14):
    city_data = data[data['city_name'] == city_name].sort_values('datetime')
    city_data = city_data[['datetime', 'main.aqi', 'components.co', 'components.no2', 'components.o3', 'components.so2', 'components.pm2_5', 'components.pm10']].dropna()
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(city_data.iloc[:, 1:])
    X, y = [], []
    for i in range(len(scaled_data) - sequence_length):
        X.append(scaled_data[i:i+sequence_length])
        y.append(scaled_data[i+sequence_length])
    X = np.array(X)
    y = np.array(y)
    return X, y, scaler, city_data

def reshape_input(X):
    return X.reshape((X.shape[0], X.shape[1], X.shape[2]))

def create_cnn_lstm_model(input_shape):
    model = Sequential()
    model.add(Conv1D(32, 2, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(2))
    model.add(Flatten())
    model.add(Reshape((-1, 32)))
    model.add(LSTM(50, activation='relu'))
    model.add(Dense(7))
    model.compile(optimizer='adam', loss='mse')
    return model

@anvil.server.callable
def train_model(city_name):
    global model, scaler
    X, y, scaler, _ = preprocess_data(city_name, data)
    X_reshaped = reshape_input(X)
    input_shape = (X_reshaped.shape[1], X_reshaped.shape[2])
    model = create_cnn_lstm_model(input_shape)
    model.fit(X_reshaped, y, epochs=10, batch_size=32, validation_split=0.2)
    return {"status": "Model trained successfully for city: " + city_name}

@anvil.server.callable
def get_predictions(city_name):
    global model, scaler
    sequence_length = 14
    future_days = 7
    X, _, _, city_data = preprocess_data(city_name, data, sequence_length)
    X_reshaped = reshape_input(X)
    latest_sequence = X_reshaped[-1].reshape(1, sequence_length, -1)
    predictions = []
    for _ in range(future_days):
        prediction = model.predict(latest_sequence)
        predictions.append(prediction[0])
        latest_sequence = np.append(latest_sequence[:, 1:, :], prediction.reshape(1, 1, -1), axis=1)
    predictions = scaler.inverse_transform(predictions)
    today_prediction = predictions[0]
    future_predictions = predictions
    return {
        "today_prediction": today_prediction.tolist(),
        "future_predictions": future_predictions.tolist()
    }

# Keep the server running to accept Anvil calls
anvil.server.wait_forever()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45228 sha256=5dac98a39e6b41d619d09a9de5c59f8264a61f85bce52279650afb83f1e2baeb
  Stored in directory: /root/.cache/pip/wheels/2e/7c/ad/d9c746276bf024d44296340869fcb169f1e5d80fb147351a57
Successfully built ws4py


Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER
Epoch 1/10
39/39 [==============================] - 3s 21ms/step - loss: 0.0926 - val_loss: 0.0368
Epoch 2/10
39/39 [==============================] - 0s 11ms/step - loss: 0.0166 - val_loss: 0.0286
Epoch 3/10
39/39 [==============================] - 0s 10ms/step - loss: 0.0149 - val_loss: 0.0263
Epoch 4/10
39/39 [==============================] - 0s 10ms/step - loss: 0.0141 - val_loss: 0.0233
Epoch 5/10
39/39 [==============================] - 0s 10ms/step - loss: 0.0136 - val_loss: 0.0231
Epoch 6/10
39/39 [==============================] - 0s 11ms/step - loss: 0.0129 - val_loss: 0.0227
Epoch 7/10
39/39 [==============================] - 0s 11ms/step - loss: 0.0129 - val_loss: 0.0218
Epoch 8/10
39/39 [==============================] - 1s 13ms/step - loss: 0.0126 - val_loss: 0.0237
Epoch 9/10
39/39 [==============================] - 0s 8ms/step - loss: 0.0127 - val_loss: 0.0232
Epoch

In [ ]:
!pip install anvil-uplink pandas scikit-learn tensorflow

import anvil.server
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Reshape, LSTM, Dense

# Connect to Anvil
anvil.server.connect("")

# Load the data directly from the Colab environment
data = pd.read_csv('Combined_PTF04_Revised.csv', low_memory=False)
data['datetime'] = pd.to_datetime(data['datetime'], errors='coerce')
data = data.dropna(subset=['datetime'])

# Extract unique city names
unique_city_names = list(data['city_name'].unique())

scaler = None
model = None

def preprocess_data(city_name, data, sequence_length=14):
    city_data = data[data['city_name'] == city_name].sort_values('datetime')
    city_data = city_data[['datetime', 'main.aqi', 'components.co', 'components.no2', 'components.o3', 'components.so2', 'components.pm2_5', 'components.pm10']].dropna()
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(city_data.iloc[:, 1:])
    X, y = [], []
    for i in range(len(scaled_data) - sequence_length):
        X.append(scaled_data[i:i+sequence_length])
        y.append(scaled_data[i+sequence_length])
    X = np.array(X)
    y = np.array(y)
    return X, y, scaler, city_data

def reshape_input(X):
    return X.reshape((X.shape[0], X.shape[1], X.shape[2]))

def create_cnn_lstm_model(input_shape):
    model = Sequential()
    model.add(Conv1D(32, 2, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(2))
    model.add(Flatten())
    model.add(Reshape((-1, 32)))
    model.add(LSTM(50, activation='relu'))
    model.add(Dense(7))
    model.compile(optimizer='adam', loss='mse')
    return model

@anvil.server.callable
def train_model(city_name):
    global model, scaler
    X, y, scaler, _ = preprocess_data(city_name, data)
    X_reshaped = reshape_input(X)
    input_shape = (X_reshaped.shape[1], X_reshaped.shape[2])
    model = create_cnn_lstm_model(input_shape)
    model.fit(X_reshaped, y, epochs=10, batch_size=32, validation_split=0.2)
    return {"status": "Model trained successfully for city: " + city_name}

@anvil.server.callable
def get_predictions(city_name):
    global model, scaler
    sequence_length = 14
    future_days = 7
    X, _, _, city_data = preprocess_data(city_name, data, sequence_length)
    X_reshaped = reshape_input(X)
    latest_sequence = X_reshaped[-1].reshape(1, sequence_length, -1)
    predictions = []
    for _ in range(future_days):
        prediction = model.predict(latest_sequence)
        predictions.append(prediction[0])
        latest_sequence = np.append(latest_sequence[:, 1:, :], prediction.reshape(1, 1, -1), axis=1)
    predictions = scaler.inverse_transform(predictions)
    today_prediction = predictions[0]
    future_predictions = predictions
    return {
        "today_prediction": today_prediction.tolist(),
        "future_predictions": future_predictions.tolist()
    }

# Keep the server running to accept Anvil calls
anvil.server.wait_forever()


#V3



In [ ]:
!pip install pandas


In [ ]:
import pandas as pd

# Load the Excel file
file_path = '/content/DATA_04092024.xlsx'  # Colab typically stores files in /content/
xls = pd.ExcelFile(file_path)

# Check sheet names
sheet_names = xls.sheet_names
print("Sheet Names:", sheet_names)

# Load a specific sheet into a DataFrame
df = pd.read_excel(xls, sheet_name=sheet_names[0])  # Replace 0 with the sheet index or name
print(df.head())  # Display the first few rows of the data


Sheet Names: ['Sheet1']
                    datetime  main.aqi  components.co  components.no2  \
0  2023-11-06 15:52:19+08:00         2         337.12            1.95   
1  2023-11-06 15:52:20+08:00         2         343.80            5.01   
2  2023-11-06 15:52:22+08:00         1         240.33            1.41   
3  2023-11-06 15:52:23+08:00         1         181.91            0.41   
4  2023-11-06 15:52:25+08:00         5        1001.36            5.44   

   components.o3  components.so2  components.pm2_5  components.pm10  \
0          76.53            1.10             18.51            20.02   
1          60.80            3.61              8.12             9.62   
2          51.50            0.41              1.28             1.65   
3          37.55            1.33              1.12             1.28   
4         254.63           28.85            102.82           131.67   

      city_name  
0      Alaminos  
1  Angeles City  
2      Antipolo  
3       Bacolod  
4        Bacoor  
